# Challenge: Build your own NLP model
For this challenge, you will need to choose a corpus of data from nltk or another source that includes categories you can predict and create an analysis pipeline that includes the following steps:

    1.Data cleaning / processing / language parsing
    
    2.Create features using two different NLP methods: For example, BoW vs tf-idf.
    
    3.Use the features to fit supervised learning models for each feature set to predict the category outcomes.

    4.Assess your models using cross-validation and determine whether one model performed better.

    5.Pick one of the models and try to increase accuracy by at least 5 percentage points.

Write up your report in a Jupyter notebook. Be sure to explicitly justify the choices you make throughout, and submit it below.



In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk


import warnings
warnings.filterwarnings('ignore')

In [2]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [3]:
leaves = gutenberg.raw('whitman-leaves.txt')
stories = gutenberg.raw('bryant-stories.txt')
busterbrown = gutenberg.raw('burgess-busterbrown.txt')
mobydick = gutenberg.raw('melville-moby_dick.txt')

In [4]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text


# The Chapter indicator is idiosyncratic
leaves = re.sub(r'Chapter \d+', '', leaves)
leaves = re.sub(r'[A-Z]{2,100}', '', leaves)

stories = re.sub(r'CHAPTER .*', '', stories)
stories = re.sub(r'[A-Z]{2,100}', '', stories)

busterbrown = re.sub(r'Chapter \d+', '', busterbrown)
busterbrown = re.sub(r'\w+\t', '', busterbrown)

mobydick = re.sub(r'CHAPTER .*', '', mobydick)
mobydick = re.sub(r'CHAPTER \w+', '', mobydick)


leaves = text_cleaner(leaves[:int(len(leaves)/10)])
stories = text_cleaner(stories[:int(len(stories)/10)])
busterbrown = text_cleaner(busterbrown[:int(len(busterbrown)/10)])
mobydick = text_cleaner(mobydick[:int(len(mobydick)/10)])

In [5]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [6]:
leaves = text_cleaner(leaves)
stories = text_cleaner(stories)
busterbrown = text_cleaner(busterbrown)
mobydick = text_cleaner(mobydick)

In [7]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')

leaves_doc = nlp(leaves)
stories_doc = nlp(stories)
busterbrown_doc = nlp(busterbrown)
mobydick_doc = nlp(mobydick)

In [8]:
# Group into sentences.
leaves_sents = [[sent, "Blake"] for sent in leaves_doc.sents]
stories_sents = [[sent,  "Bryant"] for sent in stories_doc.sents]
busterbrown_sents = [[sent, "Burgess"] for sent in busterbrown_doc.sents]
mobydick_sents = [[sent, "Melville"] for sent in mobydick_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(leaves_sents + stories_sents + busterbrown_sents + mobydick_sents)

In [9]:
sentences.head()

,0,1
0,"(Come, ,, said, my, soul, ,, Such, verses, for...",Blake
1,"(That, should, I, after, return, ,, Or, ,, lon...",Blake
2,"(I, Sing, One's, -, self)",Blake
3,"(I, sing, ,, a, simple, separate, person, ,, Y...",Blake
4,"(En, -, Masse, .)",Blake


# Bow

In [10]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

In [11]:
# Set up the bags.
leavesWords = bag_of_words(leaves_doc)
storiesWords = bag_of_words(stories_doc)
busterbrownWords = bag_of_words(busterbrown_doc)
mobydickWords = bag_of_words(mobydick_doc)

# Combine bags to create a set of unique words.
common_words = set(leavesWords + storiesWords + busterbrownWords + mobydickWords)

In [12]:
word_counts = bow_features(sentences, common_words)

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500
Processing row 550
Processing row 600
Processing row 650
Processing row 700
Processing row 750
Processing row 800
Processing row 850
Processing row 900
Processing row 950
Processing row 1000
Processing row 1050
Processing row 1100
Processing row 1150
Processing row 1200
Processing row 1250
Processing row 1300
Processing row 1350
Processing row 1400
Processing row 1450
Processing row 1500
Processing row 1550
Processing row 1600
Processing row 1650
Processing row 1700
Processing row 1750
Processing row 1800
Processing row 1850
Processing row 1900
Processing row 1950
Processing row 2000
Processing row 2050
Processing row 2100
Processing row 2150
Processing row 2200
Processing row 2250
Processing row 2300
Processing row 2350
Processing row 2400
Processing row 2450
Processing row 2500
Pro

In [13]:
word_counts.head()

,suicide,"10,440",musical,inventor,os,orbic,convex,lovely,Till,Coffin,...,industry,fate,chair,desire,Librarian,physiognomy,look'd,til,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Come, ,, said, my, soul, ,, Such, verses, for...",Blake
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(That, should, I, after, return, ,, Or, ,, lon...",Blake
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, Sing, One's, -, self)",Blake
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, sing, ,, a, simple, separate, person, ,, Y...",Blake
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(En, -, Masse, .)",Blake


# Supervised Learning on BOWs features

In [14]:
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from xgboost import XGBClassifier

In [15]:
from sklearn.model_selection import train_test_split
Y_bow = word_counts['text_source']
X_bow = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bow, 
                                                    Y_bow,
                                                    test_size=0.3,
                                                    random_state=99)

In [16]:
#Let's see the accuracy scores for all of our models. 
models = []
results = []
names = []
print('\t BOWs Accuracy\n\t score\t  std')

models.append(('NB', GaussianNB()))

models.append(('KNN', KNeighborsClassifier()))

models.append(('LR', LogisticRegression()))

models.append(('SVM', SVC(kernel='linear')))

models.append(('RF', RandomForestClassifier()))

models.append(('DT', DecisionTreeClassifier()))

models.append(('GB', GradientBoostingClassifier()))
models.append(('LRTuned', LogisticRegression(C=10,penalty= 'l2')))

for name, model in models:
    kfold = model_selection.KFold(n_splits = 5, random_state = 99)
    cv_results = model_selection.cross_val_score(model, X_train_bow, y_train_bow,
                                                cv = 5, 
                                                 scoring = 'accuracy')
   

    msg = "%s:\t%.4f (%.4f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

	 BOWs Accuracy
	 score	  std
NB:	0.5865 (0.0352)
KNN:	0.5383 (0.0561)
LR:	0.7356 (0.0130)
SVM:	0.7123 (0.0150)
RF:	0.6957 (0.0165)
DT:	0.6585 (0.0213)
GB:	0.6973 (0.0152)
LRTuned:	0.7467 (0.0076)


So when using BOWs Logistic Regression explains the variance at 73.50%.

# TFIDF on sentences

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer1=TfidfVectorizer(max_df=0.5, 
                            min_df=2, 
                            stop_words='english',
                            lowercase=False,
                            use_idf=True,
                            #norm=u'13',
                            smooth_idf=True)

In [18]:
#Take pandas series (text_sentence), convert from spacy object to string
sentence1 = word_counts['text_sentence'].astype(str)
print(type(sentence1))

#Pass pandas series to our vectorizer model
text_tfidf = vectorizer1.fit_transform(sentence1)
print(type(text_tfidf))


<class 'pandas.core.series.Series'>
<class 'scipy.sparse.csr.csr_matrix'>


In [19]:
#List of features
terms = vectorizer1.get_feature_names()

#Shape
n = text_tfidf.shape[0]

#A list of dictionaries, one per paragraph
tfidf_bysent = [{} for _ in range(0,n)]

#for each sentence, lists the feature words and their tf-idf scores
for i, j in zip(*text_tfidf.nonzero()):
    tfidf_bysent[i][terms[j]] = text_tfidf[i, j]

#Show first dictionary
display(tfidf_bysent[3])
print(type(tfidf_bysent))

{'sing': 0.2776142230122457,
 'simple': 0.3507970589256886,
 'separate': 0.34031646700127266,
 'person': 0.31824136686009585,
 'Yet': 0.30328425255854324,
 'utter': 0.36430889194788424,
 'word': 0.5983950156333977}

<class 'list'>


In [20]:
#Create dataframe for this feature set
tfidf_df = pd.DataFrame(columns=terms)
tfidf_df['text_sentence'] = word_counts['text_sentence']
tfidf_df['text_source'] = word_counts['text_source']
tfidf_df.loc[:, terms] = 0

In [21]:
Y_tfidf = tfidf_df['text_source']
X_tfidf = tfidf_df.drop(['text_sentence','text_source'], 1)

X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, 
                                                    Y_tfidf,
                                                    test_size=0.3,
                                                    random_state=99)

# Supervised learning on TDFIDF features

In [22]:
#Let's see the accuracy scores for all of our models. 
models1 = []
results1 = []
names1 = []
print('\t TFIDF Accuracy\n\t score\t  std')

models1.append(('NB', GaussianNB()))

models1.append(('KNN', KNeighborsClassifier()))

models1.append(('LR', LogisticRegression(C=1e9)))

models1.append(('SVM', SVC(kernel='linear')))

models1.append(('RF', RandomForestClassifier()))

models1.append(('DT', DecisionTreeClassifier()))

models1.append(('GB', GradientBoostingClassifier()))
models1.append(('LRTuned', LogisticRegression(C=10,penalty= 'l2')))

for name1, model1 in models1:
    kfold1 = model_selection.KFold(n_splits = 5, random_state = 99)
    cv_results1 = model_selection.cross_val_score(model1, X_train_tfidf, y_train_tfidf,
                                                cv = 5, 
                                                 scoring = 'accuracy')
   

    msg1 = "%s:\t%.4f (%.4f)" % (name1, cv_results1.mean(), cv_results1.std())
    print(msg1)

	 TFIDF Accuracy
	 score	  std
NB:	0.2533 (0.0008)
KNN:	0.5405 (0.0017)
LR:	0.5405 (0.0017)
SVM:	0.5405 (0.0017)
RF:	0.5405 (0.0017)
DT:	0.5405 (0.0017)
GB:	0.5405 (0.0017)
LRTuned:	0.5405 (0.0017)


So Logistic Regression will be the model to try and increase the accuracy.

In [23]:
#import GridseachCV to find the best parameters for Logistic Regression
from sklearn.model_selection import GridSearchCV

#Instantiate
clf = LogisticRegression()

#Grid
parameter_grid = {'C': [0.01, 0.1, 1, 2, 10, 100], 'penalty': ['l1', 'l2']}

#Gridsearch
gridsearch = GridSearchCV(clf, parameter_grid)
gridsearch.fit(X_train_bow, y_train_bow);

#Get best hyperparameters
gridsearch.best_params_

{'C': 10, 'penalty': 'l2'}

# Analysis
BOWs was better at predicting than TFIDF.  Logistic Regression was the best algorithm.  I found the best parameters using gridsearch but I was only able to increase the prediction by 1.11%. 

I will come back to this and use gridsearch for the other algorithms to see if I can increase the accuracy by more than 5%.